In [1]:
!pip install pycaret

In [18]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipaddress
from sklearn.preprocessing import MinMaxScaler
from pycaret.anomaly import *
# from pycaret.classification import *

In [19]:
# data = pd.read_csv('ics_training.csv')
data = pd.read_csv('os_training.csv')

In [20]:
def mac_to_int(mac):
    return int(mac.replace(':', ''), 16)
def ip_to_int(ip):
    return int(ipaddress.ip_address(ip))
def hex_to_int(x):
    return int(x, 16)
def proto_to_cat(p):
    protocols = ["Modbus/TCP","TCP","HTTP","TLSv1.3","TLSv1.2",
     "TPKT","NBSS","SSLv3","HTTP/XML","SMB","TLSv1",
     "OCSP","TLSv1.1","SSHv2","Portmap","DCERPC","DNS",
     "Gryphon","RDP","IRC","SSH","SMB2","SSLv2","SIP",
     "RSL","GIOP","COTP","PMPROXY","H1"]
    return protocols.index(p)+1

# data = data.drop(['date','time','ip.dsfield','ip.frag_offset','ip.version','eth.type','ip.hdr_len'], axis = 1)
# data = data.drop(['date','time','ip.dsfield','ip.frag_offset','ip.version','eth.type','ip.hdr_len','ip.proto','ip.len','tcp.hdr_len','tcp.flags','tcp.time_delta','ip.src','ip.dst','ip.id','ip.flags','tcp.seq','tcp.ack','raw'], axis = 1)
data = data.drop(['date','ip.dsfield','ip.frag_offset','ip.version','eth.type','ip.hdr_len','ip.proto','ip.len','tcp.hdr_len','tcp.flags','tcp.time_delta','ip.src','ip.dst','ip.id','ip.flags','tcp.seq','tcp.ack','raw'], axis = 1)
data['time'] = data['time'].apply(lambda time : sum(x * int(t) for x, t in zip([3600, 60, 1], str(time).split(":"))) )
data['ip.ttl'] = data['ip.ttl'].apply(lambda x : int(x))
# data['eth.src'] = data['eth.src'].apply(mac_to_int)
# data['eth.dst'] = data['eth.dst'].apply(mac_to_int)
# data['ip.src'] = data['ip.src'].apply(ip_to_int)
# data['ip.dst'] = data['ip.dst'].apply(ip_to_int)
# data['ip.len'] = data['ip.len'].apply(int)
# data['ip.id'] = data['ip.id'].apply(hex_to_int)
# data['ip.flags'] = data['ip.flags'].apply(hex_to_int)
# data['tcp.flags'] = data['tcp.flags'].apply(hex_to_int)
# data['ip.proto'] = data['ip.proto'].apply(proto_to_cat)
# data = data.drop(['raw'],axis=1)

In [21]:
print(data['attack'].value_counts())
y = data['attack']
X = data.drop(['attack'],axis=1)
print(data.info())
# for col in data.columns:
#     print(data[col].value_counts())
# scaling = MinMaxScaler(feature_range=(-1,1)).fit(data)
# data = scaling.transform(data)
# data = pd.DataFrame(data, columns = ['eth.src','eth.dst','ip.src','ip.dst','ip.len','ip.id','ip.flags','ip.ttl','ip.proto','tcp.srcport','tcp.dstport','tcp.seq','tcp.ack','tcp.hdr_len','tcp.flags','tcp.window_size','tcp.time_delta','tcp.time_relative'])


0    12564
1     3835
Name: attack, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16399 entries, 0 to 16398
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   time               16399 non-null  int64  
 1   eth.src            16399 non-null  object 
 2   eth.dst            16399 non-null  object 
 3   ip.ttl             16399 non-null  int64  
 4   tcp.srcport        16399 non-null  int64  
 5   tcp.dstport        16399 non-null  int64  
 6   tcp.window_size    16399 non-null  int64  
 7   tcp.time_relative  16399 non-null  float64
 8   attack             16399 non-null  int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 1.1+ MB
None


In [22]:
# anomaly
setup(data,session_id = 123)
# class
# setup(data, target = 'attack',fix_imbalance = True)

,Description,Value
0,session_id,123
1,Original Data,"(16399, 9)"
2,Missing Values,False
3,Numeric Features,5
4,Categorical Features,4
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(16399, 17)"
9,CPU Jobs,-1


(None,
 KFold(n_splits=10, random_state=None, shuffle=False),
 <MLUsecase.ANOMALY: 4>,
 {'USI',
  'X',
  '_all_metrics',
  '_all_models',
  '_all_models_internal',
  '_available_plots',
  '_gpu_n_jobs_param',
  '_internal_pipeline',
  '_ml_usecase',
  'create_model_container',
  'dashboard_logger',
  'data_before_preprocess',
  'display_container',
  'exp_name_log',
  'experiment__',
  'fix_imbalance_method_param',
  'fix_imbalance_param',
  'fold_generator',
  'fold_groups_param',
  'fold_groups_param_full',
  'fold_param',
  'fold_shuffle_param',
  'gpu_param',
  'html_param',
  'imputation_classifier',
  'imputation_regressor',
  'iterative_imputation_iters_param',
  'log_plots_param',
  'logging_param',
  'master_model_container',
  'n_jobs_param',
  'prep_pipe',
  'pycaret_globals',
  'seed',
  'stratify_param',
  'target_param',
  'transform_target_method_param',
  'transform_target_param'},
 Pipeline(memory=None, steps=[('empty_step', 'passthrough')], verbose=False),
 'UNSUPERVI

In [ ]:
# anomaly
# models()
# class
# model = compare_models()

In [25]:
#for anomaly detection
cluster = create_model('cluster', fraction = 0.23)
cluster_result = assign_model(cluster)
cluster_anomaly=cluster_result[cluster_result['Anomaly']==1]
cluster_normal=cluster_result[cluster_result['Anomaly']==0]

In [29]:
svm = create_model('svm', fraction = 0.3)
svm_result = assign_model(cluster)
svm_anomaly=svm_result[svm_result['Anomaly']==1]
svm_normal=svm_result[svm_result['Anomaly']==0]

In [30]:
print(svm_anomaly['attack'].value_counts())
print(svm_normal['attack'].value_counts())

1    2263
0    1501
Name: attack, dtype: int64
0    11063
1     1572
Name: attack, dtype: int64


In [16]:
# for classification
svm = create_model('svm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9408,0.0000,0.9699,0.8113,0.8836,0.8443,0.8503
1,0.9233,0.0000,0.9850,0.7572,0.8562,0.8052,0.8181
2,0.9373,0.0000,0.9887,0.7922,0.8796,0.8379,0.8473
3,0.9225,0.0000,0.9887,0.7536,0.8553,0.8036,0.8175
4,0.9425,0.0000,0.9812,0.8106,0.8878,0.8496,0.8566
5,0.9721,0.0000,0.9586,0.9239,0.9410,0.9227,0.9230
6,0.9521,0.0000,0.9738,0.8442,0.9043,0.8726,0.8766
7,0.9521,0.0000,0.9700,0.8464,0.9040,0.8723,0.8759
8,0.9756,0.0000,0.9888,0.9135,0.9496,0.9336,0.9349


In [17]:
importance = svm.coef_[0]
print(importance)
# summarize feature importance
for i,v in enumerate(importance):
 print('Feature: '+features_names[i]+', Score: %.5f' %v)
# plot feature importance
plt.bar([x for x in features_names], importance)
plt.show()

[-12279.3035349    8305.01232715   8930.55653853    540.55005902
   3145.65271413    283.3819065     -13.01632889   -268.55775416
     39.88371008    -52.90003897      0.        ]


NameError: name 'features_names' is not defined